In [85]:
# Import libraries
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model

Add and adjust 'Justin.csv' including pre-proccessing data like time

In [86]:
# Load the dataset without explicit date parsing
file_path = 'justin.csv'
df = pd.read_csv(file_path)
# Preprocess Date & Time
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d %H:%M')
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Verify Data
print(df.head())


  Location            DateTime
0   Stever 1900-06-30 23:09:00
1   Stever 1900-07-01 22:34:00
2   Stever 1900-07-01 16:00:00
3   Stever 1900-07-01 22:46:00
4     Cafe 1900-07-02 12:25:00


Setup Time and Date in pandas correctly

In [87]:
# Extract features from datetime
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.dayofweek

# Encoding via OneHotEncoder
label_encoder = LabelEncoder()
df['Location_Code'] = label_encoder.fit_transform(df['Location'])

# Prepare features (X) and target (y)
X = df[['Hour', 'DayOfWeek']].values
y = df['Location_Code'].values

print(df.head())

  Location            DateTime  Hour  DayOfWeek  Location_Code
0   Stever 1900-06-30 23:09:00    23          5              2
1   Stever 1900-07-01 22:34:00    22          6              2
2   Stever 1900-07-01 16:00:00    16          6              2
3   Stever 1900-07-01 22:46:00    22          6              2
4     Cafe 1900-07-02 12:25:00    12          0              0


Setup OneHotEncoder

In [88]:
# One-hot encode categorical features (DayOfWeek)
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],
    remainder='passthrough'
)
X = ct.fit_transform(X)

Splitting dataset into training and testing

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standarization via scaling

In [90]:
scaler = StandardScaler()
sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Defining model

In [99]:
# Set input shape in friendly way 
# to be adjustable depending on # of vars
input_shape = (7,) 

# Input layer
inputs = Input(shape=input_shape)

# Hidden layers
x = Dense(32, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)

# Output layer
outputs = Dense(4, activation='relu')(x)# Example output layer 
outputs = Dense(3, activation='relu')(x)# for binary classification

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


Model Summary - find how many params used

In [100]:
model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_20 (InputLayer)     │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 835 (3.26 KB)

 Trainable params: 835 (3.26 KB)

 Non-trainable params: 0 (0.00 B)

In [101]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.1667 - loss: 1.2431 - val_accuracy: 0.0000e+00 - val_loss: 1.1745
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step - accuracy: 0.1667 - loss: 1.2185 - val_accuracy: 0.2000 - val_loss: 1.1562
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1667 - loss: 1.1965 - val_accuracy: 0.2000 - val_loss: 1.1399
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step - accuracy: 0.4444 - loss: 1.1764 - val_accuracy: 0.2000 - val_loss: 1.1259
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step - accuracy: 0.4444 - loss: 1.1550 - val_accuracy: 0.2000 - val_loss: 1.1152
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step - accuracy: 0.4444 - loss: 1.1334 - val_accuracy: 0.2000 - val_loss: 1.1059
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step - accuracy: 0.4444 - loss: 1.1120 - val_accuracy: 0.4000 - val_loss: 1.0972
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - accuracy: 0.5000 - loss: 1.0908 - val_accuracy: 0.4000 - val_loss: 1

Accuracy of 77.78% currently, simply need more data in order to make it better but for now this isn't bad, it also may be overfitting so keep that in mind

Save Model File

In [103]:
filePath = '/users/alexanderokenczyc/documents/github/trollModelRA'
model.save('justin.keras')
model.save('justin.h5')
#model.save('trollModel.h5')
# tf.keras.Model.save('trollModel.h5', filepath=filepath)